<a href="https://colab.research.google.com/github/saleh-imran/BigDataProcessing/blob/main/Movie_Recommendation_with_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import csv

# Replace 'input_file.txt' with the path to your input text file.
input_file = '/content/drive/MyDrive/data.txt'
output_file = 'data.csv'  # Replace 'output_file.csv' with your desired output file name.

# Read the data from the input text file and convert it to the desired format.
data = []
with open(input_file, 'r') as file:
    for line in file:
        # Split the line by tabs and extract UserID, MovieID, and rating.
        user_id, movie_id, rating, _ = line.strip().split('\t')
        data.append([user_id, movie_id, rating])

# Add header to the data
header = ['UserID', 'MovieID', 'rating']
data.insert(0, header)

# Write the data into a CSV file.
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("Data converted and saved to", output_file)


Data converted and saved to data.csv


**Installing Pyspark**

In [43]:
pip install pyspark

**Importing Libraries**

In [44]:
from pyspark.sql import SparkSession
from pyspark.mllib.recommendation import ALS, Rating

**Setting up Spark Session**

In [45]:
# Step 1: Set up the SparkSession
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

**Load Data**

In [46]:
# Step 2: Load and parse the data
data_path = "/content/data.csv"  # Replace with the path to your dataset file
data = spark.read.csv(data_path, header=True, inferSchema=True)
ratings = data.rdd.map(lambda row: Rating(int(row["UserID"]), int(row["MovieID"]), float(row["rating"])))

**Build Model**

In [47]:
# Step 3: Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

**Evaluate Model**

In [48]:

# Step 4: Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.4816340849785578
